In [1]:
import pandas as pd

In [2]:
# Read in the processed raw data
reddit_df = pd.read_csv('../../combined_data.csv')

In [3]:
reddit_df.head(1)

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation,yearmonth,title,index
0,SUTD entry requirements is more or less the sa...,2020-01-01 00:00:33,Twrd4321,/r/singapore/comments/eia0ap/sit_suss_or_sutd/...,t3_eia0ap,t1_fcom49y,fconqf5,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",2020-01,sit suss or sutd,0


In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

c:\Users\65835\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\65835\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\65835\.cache\huggingface\hub\models--Hate-speech-CNERG--bert-base-uncased-hatexplain-rationale-two. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Dev

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

Some weights of the model checkpoint at Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two were not used when initializing BertForSequenceClassification: ['bert_pooler.dense.bias', 'bert_pooler.dense.weight', 'token_classifier.bias', 'token_classifier.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight